In [30]:
import numpy as np
import scipy.linalg as linalg
from dtuimldmtools import similarity
import matplotlib.pyplot as plt
from scipy.linalg import svd
from pandas.plotting import scatter_matrix
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pyplot import figure, legend, plot, show, xlabel, ylabel
# exercise 8.1.1
import torch
import importlib_resources
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pylab import (
    figure,
    grid,
    legend,
    loglog,
    semilogx,
    show,
    subplot,
    title,
    xlabel,
    ylabel,
)
from scipy.io import loadmat
from sklearn import model_selection
import importlib_resources
import matplotlib.pyplot as plt
import numpy as np
import torch
from scipy import stats
from scipy.io import loadmat
from sklearn import model_selection
from dtuimldmtools import draw_neural_net, train_neural_net


from dtuimldmtools import rlr_validate

# fetch dataset 
# wine = fetch_ucirepo(id=109) 
  
# # data (as pandas dataframes) 
# X = wine.data.features 
# y = wine.data.targets 

# totaldata= (wine.data)
  
# metadata 
# print(wine.metadata) 
  
# variable information 
# print(wine.variables) 

# OFFLINE LOADING OF DATA
X = np.loadtxt('../wine/wine.data', delimiter=',')
y = X[:,0]
X = np.delete(X,0,axis=1)
# y = np.loadtxt('./wine/wine.names', delimiter=',')

Xorig = X
# Standardizing the data

X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# One of K encoding
# y = similarity.one_of_k(y)

# Ensure y values are within the range of the identity matrix's size
num_classes = int(np.max(y)) + 1
y2 = np.eye(num_classes)[y.astype(int)]

#removing first column of y
y2 = y2[:,1:]

#Appending y to X
X = np.append(X,y2,axis=1)

#Extracting the first column of X
y = X[:,0]



attributeNames = [
    "Alcohol",
    "Malic acid",
    "Ash",
    "Alcalinity of ash",
    "Magnesium",
    "Total phenols",
    "Flavanoids",
    "Nonflavanoid phenols",
    "Proanthocyanins",
    "Color intensity",
    "Hue",
    "OD280/OD315 of diluted wines",
    "Proline"
]

#Removing the first column of X
X = np.delete(X,0,axis=1)
# Removing alcohol from attributenames
attributeNames = attributeNames[1:]
N, M = X.shape

# Add offset attribute
X = np.concatenate((np.ones((X.shape[0], 1)), X), 1)
attributeNames = ["Offset"] + attributeNames + ["Class 1", "Class 2", "Class 3"]
M = M + 1

## Crossvalidation
# Create crossvalidation partition for evaluation
K = 10



In [ ]:

#Generalization error
#two-fold K=10 fold cross validation 
# K = 10
# N = X.shape[0]
# M = X.shape[1]
# #splitting the data into K folds
# fold_size = N//K
# np.random.shuffle(X)
# folds = np.split(X, K)


from sklearn.metrics import mean_squared_error


CV = model_selection.KFold(K, shuffle=True)

# Creating a outer cross-validation
# Outer cross-validation
nested_cv = model_selection.KFold(n_splits=K, shuffle=True, random_state=1)

#CV = model_selection.KFold(K, shuffle=False)

# Values of lambda
lambdas = np.power(10.0, range(-4, 10))

# Initialize variables
# T = len(lambdas)
Error_train = np.empty((K, 1))
Error_test = np.empty((K, 1))
Error_train_rlr = np.empty((K, 1))
Error_test_rlr = np.empty((K, 1))
Error_train_nofeatures = np.empty((K, 1))
Error_test_nofeatures = np.empty((K, 1))
w_rlr = np.empty((M, K))
mu = np.empty((K, M - 1))
sigma = np.empty((K, M - 1))
w_noreg = np.empty((M, K))
best_hidden_units = []
best_lambdas = []

# Initialize arrays to store errors for each fold
baseline_errors_train = np.empty(K)
baseline_errors_test = np.empty(K)

outer_errors = []
# For ANN
# Parameters for neural network classifier
outer_best_hidden_unit =[]


# Setup figure for display of learning curves and error rates in fold
summaries, summaries_axes = plt.subplots(1, 2, figsize=(10, 5))
# Make a list for storing assigned color of learning curve for up to K=10
color_list = [
    "tab:orange",
    "tab:green",
    "tab:purple",
    "tab:brown",
    "tab:pink",
    "tab:gray",
    "tab:olive",
    "tab:cyan",
    "tab:red",
    "tab:blue",
]
# Define the model
# model = lambda: torch.nn.Sequential(
#     torch.nn.Linear(M, n_hidden_units),  # M features to n_hidden_units
#     torch.nn.Tanh(),  # 1st transfer function,
#     torch.nn.Linear(n_hidden_units, 1),  # n_hidden_units to 1 output neuron
#     # no final tranfer function, i.e. "linear output"
# )
# loss_fn = torch.nn.MSELoss()  # notice how this is now a mean-squared-error loss

# print("Training model of type:\n\n{}\n".format(str(model())))
errors = []  # make a list for storing generalizaition error in each loop

internal_cross_validation = 10

internal_cv = model_selection.KFold(internal_cross_validation, shuffle=True, random_state=42)


k = 0

for train_index, test_index in CV.split(X, y):
    # extract training and test set for current CV fold
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    internal_cross_validation = 10

    (
        opt_val_err,
        opt_lambda,
        mean_w_vs_lambda,
        train_err_vs_lambda,
        test_err_vs_lambda,
    ) = rlr_validate(X_train, y_train, lambdas, internal_cross_validation)

    # Standardize outer fold based on training set, and save the mean and standard
    # deviations since they're part of the model (they would be needed for
    # making new predictions) - for brevity we won't always store these in the scripts
    mu[k, :] = np.mean(X_train[:, 1:], 0)
    sigma[k, :] = np.std(X_train[:, 1:], 0)

    X_train[:, 1:] = (X_train[:, 1:] - mu[k, :]) / sigma[k, :]
    X_test[:, 1:] = (X_test[:, 1:] - mu[k, :]) / sigma[k, :]

    Xty = X_train.T @ y_train
    XtX = X_train.T @ X_train

    # Compute mean squared error without using the input data at all
    Error_train_nofeatures[k] = (
        np.square(y_train - y_train.mean()).sum(axis=0) / y_train.shape[0]
    )
    Error_test_nofeatures[k] = (
        np.square(y_test - y_test.mean()).sum(axis=0) / y_test.shape[0]
    )

    # Estimate weights for the optimal value of lambda, on entire training set
    lambdaI = opt_lambda * np.eye(M)
    lambdaI[0, 0] = 0  # Do no regularize the bias term
    w_rlr[:, k] = np.linalg.solve(XtX + lambdaI, Xty).squeeze()
    # Compute mean squared error with regularization with optimal lambda
    Error_train_rlr[k] = (
        np.square(y_train - X_train @ w_rlr[:, k]).sum(axis=0) / y_train.shape[0]
    )
    Error_test_rlr[k] = (
        np.square(y_test - X_test @ w_rlr[:, k]).sum(axis=0) / y_test.shape[0]
    )

    # Estimate weights for unregularized linear regression, on entire training set
    w_noreg[:, k] = np.linalg.solve(XtX, Xty).squeeze()
    # Compute mean squared error without regularization
    Error_train[k] = (
        np.square(y_train - X_train @ w_noreg[:, k]).sum(axis=0) / y_train.shape[0]
    )
    Error_test[k] = (
        np.square(y_test - X_test @ w_noreg[:, k]).sum(axis=0) / y_test.shape[0]
    )
    # OR ALTERNATIVELY: you can use sklearn.linear_model module for linear regression:
    # m = lm.LinearRegression().fit(X_train, y_train)
    # Error_train[k] = np.square(y_train-m.predict(X_train)).sum()/y_train.shape[0]
    # Error_test[k] = np.square(y_test-m.predict(X_test)).sum()/y_test.shape[0]

    best_lambdas.append(opt_lambda)
    
    
    
    #_____________________________________________________________________________
    
    # Now we want to make a ANN to model the data
    max_hidden_units = 5  # number of hidden units
    n_replicates = 1  # number of networks trained in each k-fold
    max_iter = 2000
    print("\nCrossvalidation fold: {0}/{1}".format(k + 1, K))
    loss_fn = torch.nn.MSELoss()

    for train_inner_idx, val_inner_idx in internal_cv.split(X_train, y_train):
        # Extract training and test set for current CV fold, convert to tensors
        X_train_inner = torch.Tensor(X[train_inner_idx, :])
        y_train_inner = torch.Tensor(y[train_inner_idx])
        X_test_inner = torch.Tensor(X[val_inner_idx, :])
        y_test_inner = torch.Tensor(y[val_inner_idx])
        error_log = []  # make a list for storing generalizaition error in each loop
        for n_hidden_units in range(1, max_hidden_units):
            
            model = lambda: torch.nn.Sequential(
                torch.nn.Linear(M, n_hidden_units),  
                torch.nn.Tanh(),    
                torch.nn.Linear(n_hidden_units, 1)   
            )
            
            net, _, _ = train_neural_net(
                model,
                loss_fn,
                X=X_train_inner,
                y=y_train_inner,
                n_replicates=n_replicates,
                max_iter=max_iter,
            )

            # print("\n\tBest loss: {}\n".format(final_loss))

            # Determine estimated value for test set
            y_test_inner_est = net(X_test_inner)

            # Determine errors
            se = (y_test_inner_est.float() - y_test_inner.float()) ** 2  # squared error
            mse = (sum(se).type(torch.float) / len(y_test_inner)).data.numpy()  # mean

            error_log.append(np.mean(mse))
        # Determine errors for every hidden unit
        # store error rate for current CV fold
        best_hidden_unit = range(1, max_hidden_units)[np.argmin(error_log)]
        best_hidden_units.append(best_hidden_unit)
        errors.append((error_log[np.argmin(error_log)]))  
        
        
        
    # Outer ANN
    #_____________________________________________________________________________
    X_train = torch.Tensor(X[train_index, :])
    y_train = torch.Tensor(y[train_index])
    X_test = torch.Tensor(X[test_index, :])
    y_test = torch.Tensor(y[test_index])
    best_hidden_unit_inner = best_hidden_units[np.argmin(errors)]    
    outer_best_hidden_unit.append(best_hidden_units[np.argmin(errors)])  #The best hidden unit for the outer loop
    loss_fn = torch.nn.MSELoss()    
    model = lambda: torch.nn.Sequential(
                torch.nn.Linear(M, n_hidden_units),  
                torch.nn.Tanh(),    
                torch.nn.Linear(n_hidden_units, 1)   
            )
    # Train the net on training data
    net, final_loss, learning_curve = train_neural_net(
                model,
                loss_fn,
                X=X_train,
                y=y_train,
                n_replicates=n_replicates,
                max_iter=max_iter,
    )

    # print("\n\tBest loss: {}\n".format(final_loss))

    # Determine estimated value for test set
    y_test_est = net(X_test)

    # Determine errors
    se = (y_test_est.float() - y_test.float()) ** 2  # squared error
    mse = (sum(se).type(torch.float) / len(y_test)).data.numpy()  # mean
    outer_errors.append(mse)

    
    
    
    # BASELINE MODEL
    #_____________________________________________________________________________
    
    X_train, y_train = X[train_index, :], y[train_index]
    X_test, y_test = X[test_index, :], y[test_index]

    # Step 3: Baseline model - predict the mean of y_train
    mean_y_train = np.mean(y_train)

    # Predict the mean for both training and test sets
    y_pred_train = np.full(len(y_train), mean_y_train)
    y_pred_test = np.full(len(y_test), mean_y_train)

    # Step 4: Calculate Mean Squared Error (MSE) for training and test sets
    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_test = mean_squared_error(y_test, y_pred_test)

    # Store the errors
    baseline_errors_train[k] = mse_train
    baseline_errors_test[k] = mse_test
    
    
    
    
    k += 1
    
    
    
# THE ERRORS FOR THE LAMBDAS ARE:
print("The errors for the regularized logistic regression is:" ,Error_train_rlr)
print("and the best lambdas are: ", best_lambdas)
# THE MSE FOR THE ANN ARE:
print("outer best hidden units",outer_best_hidden_unit)
print("outer errors",outer_errors)
meanErrors = [np.mean(error) for error in errors]
print("The errors for the ANN are:" ,meanErrors)
print("and the best hidden units are: ", best_hidden_units)
# Take the mean of every array in errors

#The MSE FOR THE BASELINE
print("The errors for the baseline are:" ,baseline_errors_test)







Crossvalidation fold: 1/10

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0653058	5.0355407e-06
		Final loss:
		1622	1.0637062	8.9655725e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0642213	3.4724687e-06
		Final loss:
		1277	1.0636415	8.966117e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0683911	1.2942918e-05
		2000	1.0636047	1.2328834e-06
		Final loss:
		2000	1.0636047	1.2328834e-06

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0646794	5.598334e-06
		Final loss:
		1542	1.0631195	8.9705196e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0494462	2.839806e-06
		Final loss:
		1270	1.0489432	9.091755e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0514016	8.957043e-06
		Final loss:
		1636	1.0488038	9.092963e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		972	1.0486699	9.094124e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0540192	9.952677e-06
		2000	1.0493871	1.7039816e-06
		Final loss:
		2000	1.0493871	1.7039816e-06

	Replicate: 1

/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([144])) that is different to the input size (torch.Size([144, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


		1000	1.0641013	2.12853e-06
		Final loss:
		1290	1.0636231	8.966273e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss


/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		Final loss:
		853	1.0632325	8.9695664e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0659487	5.2561695e-06
		Final loss:
		1891	1.0633768	8.9683493e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0650107	3.4698949e-06
		Final loss:
		1653	1.0637133	8.965512e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		394	1.0483656	9.0967643e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0496778	4.996945e-06
		Final loss:
		1446	1.0485506	9.095159e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0519775	8.272236e-06
		Final loss:
		1903	1.0487355	9.0935555e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0515089	8.502658e-06
		Final loss:
		1759	1.0490712	9.0906457e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0358198	5.178884e-06
		Final loss:
		1577	1.0342908	9.2205545e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0344387	1.4981249e-06
		Final loss:
		1113	1.034271	9.2207307e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.039445	9.

/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([144])) that is different to the input size (torch.Size([144, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


		1000	1.0646489	5.710464e-06
		Final loss:
		1522	1.0631514	8.9702513e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss


/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	1.0633078	2.4664523e-06
		Final loss:
		1121	1.0630844	8.9708163e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.06527	6.4904616e-06
		Final loss:
		1607	1.0634758	8.9675143e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0637338	1.6809998e-06
		Final loss:
		1175	1.0634909	8.9673875e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.050895	7.146415e-06
		Final loss:
		1635	1.0488176	7.9562386e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0494025	5.452641e-06
		Final loss:
		1285	1.0486026	9.094708e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0489963	2.0455388e-06
		Final loss:
		1197	1.0486797	9.094039e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0520821	1.1104058e-05
		Final loss:
		1773	1.0490181	9.0911055e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.0366142	4.7149247e-06
		Final loss:
		1716	1.0349398	9.2147724e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		665	1.034139	9.2219074e-07

	Replicate: 1/1
		Iter	Loss